This script aims to calculate the sea-air CO2 flux from in-situ data.

Air-sea CO2 exchange in Port Foster, was calculated across the sampled stations according to the following equation:

$F=kK^\prime\left({\mathrm{pCO}}_{2\mathrm{W}}-{\mathrm{pCO}}_{2\mathrm{A}}\right)\ \ \ \ (Eq.1)$

where F is the flux (in mol m-2 yr-1), k is the gas transfer velocity (in cm h-1), K’ is the CO2 atmospheric equilibrium solubility from moist air (in mol L-1 atm-1) and pCO2W and pCO2A denote the partial pressures of CO2 (in µatm) in equilibrium with surface water and the overlying air respectively.

In equation 1, k  was estimated according to (Wanninkhof, 2014):

$k\ =\ a\ {<U10>}^2\left(\frac{{Sc}_{CO2}}{660}\right)^{-0.5}\ \ \ \ (Eq.\ 2)$

K’ was obtained as:

$\ln{K'}=A_1+A_2\left(\frac{100}{T}\right)+A_3\ln{\left(\frac{T}{100}\right)}+A_4\left(\frac{T}{100}\right)^2+S‰[B1+B2{\left(\frac{T}{100}\right)}+B3{\left(\frac{T}{100}\right)}^2]\ \ \ \ (Eq.3) $ 

T is the temperature in Kelvin and S is the salinity expressed in ppm. Ai and Bi are the constants (moist air) for calculation of solubilities in mol L-1 atm-1: A1 = -160.7333; A2 = 215.4152; A3 = 89.8920; A4 = -1.47759; B1 = 0.029941; B2 = -0.027455; B3 = 0.0053407 (Weiss and Price, 1980). To account for the CO2 partial pressure in air, Palmer station monthly mean data for January 2021 was used, equivalent to 410.5 ppm. Finally, CO2 saturation ratios were calculated as pCO2W/ pCO2A × 100.

In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

def calculate_average_wind_speed(file_path):
    # Load the JSON data from the specified file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Extract the wind speeds and timestamps
    wind_speeds = []
    timestamps = set()  # To track unique timestamps
    
    for i, entry in enumerate(data):
        if i % 3 != 0:  # Skip the first dictionary of each 10-minute interval
            timestamp = entry['fhora']
            if timestamp not in timestamps:
                wind_speed = entry['vel']
                if wind_speed != 'NaN':  # Ensure the wind speed is a valid number
                    wind_speeds.append(float(wind_speed))
                timestamps.add(timestamp)
    
    # Calculate the average wind speed
    if wind_speeds:
        average_wind_speed = sum(wind_speeds) / len(wind_speeds)
    else:
        average_wind_speed = 0

    return average_wind_speed

def function_x(fpCO2, temperature, salinity, wind_speed, stations, anem_height):

    '''
    Gas transfer velocity calculation (k) function

    Parameters:

    fpCO2: partial pressure of CO2 in seawater
    temperature: water temperature in ºC
    salinity: water salinity in PSU
    anem_height: the height of the meteo station at which the wind veolicty is recorded
    wind_speed: wind speed data recorded at the anemometer
    stations: number of the station

    Output:
    CO2 sea-water flux
    '''
    
    # following the Equation 2: k = a * U10^2 * (ScCO2/Sc_ref)^b

    a = 0.251  # cm/h, from Wanninkhof, 2014.
    h = anem_height # effective height of the anemometer above the mean sea level
    Vh = wind_speed
    U10 = Vh/((h/10)**0.13)  
    b = -0.5
    Sc_ref = 660  # Reference Schmidt number

    # CO2 Schmidt number calculation
    # Constants for CO2 obtained from Wanninkhof, 2014 (Table 1 - Seawater)

    A = 2116.8
    B = (-136.25)
    C = 4.7353
    D = (-0.092307)
    E = 0.0007555
    t = temperature

    Sc_CO2 = A + B*t + C*t**2 + D*t**3 + E*t**4
    #print("Sc_CO2 =", round(Sc_CO2,4))

    # Calculate the gas transfer velocity for CO2 (k)
    k = a * (U10 ** 2) * ((Sc_CO2 / Sc_ref) ** b)  # cm/h

    #print("k =", round(k,4))

    A1 = -160.7333
    A2 = 215.4152
    A3 = 89.8920
    A4 = -1.47759
    B1 = 0.029941
    B2 = -0.027455
    B3 = 0.0053407
    S = salinity
    K = 273.15
    T = K + temperature

    K_prima = np.exp(A1 + A2 * (100/T) + A3 * np.log(T/100) + A4 * (T/100)**2 + S*(B1 + B2 * (T/100) + B3 * (T/100)**2))
    #print('K0 = ', K0)

    pCO2a = 410.5 # Palmer station monthly mean data for January 2021
    delta_pCO2 = fpCO2 - pCO2a

    F_umol = k * K_prima * (delta_pCO2) * 240 # umol m-2 d-1
    F_mmol = k * K_prima * (delta_pCO2) * 0.24 # mmol m-2 d-1
    F_mol = k * K_prima * (delta_pCO2) * 24 * 10**(-5)
    F_mol_y = k * K_prima * (delta_pCO2) * 24 * 365 * 10**(-5)

    return (stations, F_mol_y, F_mmol)

def process_lists_from_csv(csv_file_path, average_speed):
    # Load data from the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Extract columns into lists
    bulk_c_list = df['pCO2w'].tolist()
    temperature_C_list = df['Temp'].tolist()
    salinity_list = df['Sal'].tolist()
    stations = df['station'].tolist()
    
     # Iterate over the elements of the lists simultaneously
    fluxes_y = []
    fluxes_d = []
    stations_f = []
    for a, b, c, e in zip(bulk_c_list, temperature_C_list, salinity_list, stations):
        # Call function_x with the corresponding elements and the single average_speed
        fluxes_y.append(round(function_x(a, b, c, average_speed, e, 12)[1], 2))
        fluxes_d.append(round(function_x(a, b, c, average_speed, e, 12)[2], 2))
        stations_f.append(function_x(a, b, c, average_speed, e, 12)[0])

    return fluxes_y, fluxes_d, stations_f

# Path to wind data file
wind_speed_file = './CO2_data/CO2_WIND_DATA.json'
average_speed = calculate_average_wind_speed(wind_speed_file)
print('Average wind speed: {}'.format(average_speed))

# Path to CH4 data file
co2_data_file = './CO2_data/CO2_data.csv'
co2_fluxes = process_lists_from_csv(co2_data_file, average_speed)
for x in range(0,20):
    print('{} = {} mmol/(m^2*d) or {} mol/(m^2*y)'.format(co2_fluxes[2][x],co2_fluxes[1][x],co2_fluxes[0][x]))

# Create a DataFrame from the fluxes and stations
df = pd.DataFrame({
    'Station': co2_fluxes[2],
    'Flux (mmol/m^2*d)': co2_fluxes[1],
    'Flux (mol/m^2*y)': co2_fluxes[0]
})

# Save the DataFrame to a CSV file
output_file_path = './CO2_data/CO2_fluxes.csv'
df.to_csv(output_file_path, index=False)



Average wind speed: 7.077483146067398
St. 1-1 = 14.8 mmol/(m^2*d) or 5.4 mol/(m^2*y)
St. 1-2 = 8.32 mmol/(m^2*d) or 3.04 mol/(m^2*y)
St. 1-3 = -3.32 mmol/(m^2*d) or -1.21 mol/(m^2*y)
St. 1-4 = -8.16 mmol/(m^2*d) or -2.98 mol/(m^2*y)
St. 1-5 = -8.54 mmol/(m^2*d) or -3.12 mol/(m^2*y)
St. 1-6 = -6.98 mmol/(m^2*d) or -2.55 mol/(m^2*y)
St. 1-7 = -7.08 mmol/(m^2*d) or -2.58 mol/(m^2*y)
St. 1-8 = -30.48 mmol/(m^2*d) or -11.13 mol/(m^2*y)
St. 1-9 = -11.37 mmol/(m^2*d) or -4.15 mol/(m^2*y)
St. 1-10 = -9.45 mmol/(m^2*d) or -3.45 mol/(m^2*y)
St. 1-11 = -8.95 mmol/(m^2*d) or -3.27 mol/(m^2*y)
St. 1-12 = -10.44 mmol/(m^2*d) or -3.81 mol/(m^2*y)
St. 1-13 = -9.95 mmol/(m^2*d) or -3.63 mol/(m^2*y)
St. 1-14 = 1.38 mmol/(m^2*d) or 0.5 mol/(m^2*y)
St. 1-15 = 51.94 mmol/(m^2*d) or 18.96 mol/(m^2*y)
St. 1-16 = 4.9 mmol/(m^2*d) or 1.79 mol/(m^2*y)
St. 1-17 = -1.26 mmol/(m^2*d) or -0.46 mol/(m^2*y)
St. 1-18 = -8.6 mmol/(m^2*d) or -3.14 mol/(m^2*y)
St. 1-19 = -10.44 mmol/(m^2*d) or -3.81 mol/(m^2*y)
St. 1-20 